# Flight delays

### Packages update

We first need to update Seaborn to its latest version (0.11.1) to use some of its latest features.

In [ ]:
!conda install seaborn==0.11.1

import pandas as pd
import seaborn as sns

### Dataset loading

We load the dataset using [pandas](https://pandas.pydata.org/), and display basic information: its shape, columns and a sample of the first rows.
<p style="color:red;font-size: medium">
    <b>Insert code in the cell below:</b><br><br>
    1. Click on the cell below to position your cursor in it.<br>
    2. Click on the top-right icon that says '<i>Find and add data</i>' when you hover your mouse over it.<br>
    3. Choose the '<i>Flights-Jan2019_csv_shaped</i>' dataset and pick '<i>pandas DataFrame</i>' in the drop-down list.<br>
</p>

In [ ]:
# INSERT CODE HERE

In [ ]:
df = df_data_1 # rename to a shorter name
print(df.shape, df.dtypes, df.head(), sep="\n\n\n")

That dataset contains __583 985 rows__ and __14 columns__ with information on departure and arrival airports, flights time, ...

### Preprocessing

More complex preprocessing can be directly performed in a notebook instead of using the Data Refinery service. At the end of the notebook, we will save the resulting dataset as a new CP4D project asset.

#### Create target column

We create a `FLIGHT_STATUS` column which merges the `CANCELLED` and `DIVERTED` columns, and set flights with no issue to `ONTIME`. This will be the target column to predict.

In [ ]:
df.CANCELLED = df.CANCELLED.transform(lambda x: "CANCELLED" if x == 1.0 else "")
df.DIVERTED = df.DIVERTED.transform(lambda x: "DIVERTED" if x == 1.0 else "")
df["FLIGHT_STATUS"] = (df.CANCELLED + df.DIVERTED).replace("", "ONTIME")
df.drop(["CANCELLED", "DIVERTED"], axis=1, inplace=True)

In [ ]:
df.FLIGHT_STATUS.value_counts(normalize=True)

The dataset is logically highly unbalanced with __96.9% flights on time__.

### Visualization

We now visualize several aspects of the dataset to better understand the data and refine the preprocessing if needed.

#### Top 10 airports

In [ ]:
df_Origin = pd.DataFrame(df["ORIGIN"].value_counts().nlargest(n=10))
sns.barplot(x=df_Origin.index, y="ORIGIN", data=df_Origin)

This shows the 10 airports with most flights; the most frequented one (with more than 30 000 departure flights in January 2019) is `ATL` (for Hartsfield-Jackson International Airport in Atlanta).

#### Flights per hour

The following plot displays the number of flights according to hours of the day (as specified in the `DEP_TIME_BLK` variable).

In [ ]:
df_DepTime = pd.DataFrame(df["DEP_TIME_BLK"].value_counts())
sns.barplot(x=df_DepTime.DEP_TIME_BLK, y=df_DepTime.index, order=sorted(df_DepTime.index), palette="crest", orient="h")

#### Distribution histograms

This plot shows the distribution of variables in the dataset.

In [ ]:
df.hist(figsize=(20,10))

#### Correlation matrix

This final plot displays how variables are pair-wise dependent.

In [ ]:
sns.heatmap(df.corr())

### Preprocessing again

We can check the NaN values in the dataset, and notice that some column values are always NaN for cancelled flights: departure and arrival times, for obvious reasons. <br>
This means we can't use these columns because they are biased (we don't know the theoretical departure time of cancelled flights) and need to be excluded.

In [ ]:
df[df.FLIGHT_STATUS == "CANCELLED"].isna().mean()

We therefore drop these columns, and save that new dataset as a CP4D asset. <br>
Note that this could have been done through Data Refinery again, this is to show that both options are possible to preprocess a dataset.

In [ ]:
df.drop(["DEP_TIME", "DEP_DEL15", "ARR_TIME", "ARR_DEL15"], axis=1, inplace=True)

from project_lib import Project
Project.access().save_data("Flights-Jan2019-Clean.csv", df.to_csv(index=False))

__END__